# Redes Neuronales con Pytorch

## Introduction

En este ejercicio se implementa una red neuronal para reconocimiento de digitos utilizando pytorch.

Antes de empezar la ejecución de las partes de codigo correspondienters a los ejercicios, se requiere importar todas las librerias necesarias.

In [1]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [2]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that
# "always" works.
class NNEMNIST(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NNEMNIST, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 300)
        # self.fc2 = nn.Linear(800, 150)
        self.fc2 = nn.Linear(300, num_classes)

        # self.fc1 = nn.Linear(input_size, 47)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = torch.relu(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        # x = torch.sigmoid(x)
        # x = self.fc3(x)
        return x

In [3]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 784
num_classes = 47
learning_rate = 0.001
batch_size = 500000
num_epochs = 5

cuda


In [4]:
# Load Training and Test data
train_dataset = datasets.EMNIST(root="dataset/", split="bymerge", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.EMNIST(root="dataset/", split="bymerge", train=False, transform=transforms.ToTensor(), download=True)
print(len(train_dataset))
print(len(test_dataset))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


Extracting dataset/EMNIST/raw/gzip.zip to dataset/EMNIST/raw
697932
116323


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
for c in train_dataset:
  print(c)

Output hidden; open in https://colab.research.google.com to view.

In [5]:
# Initialize network
model = NNEMNIST(input_size=input_size, num_classes=num_classes).to(device)

In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Train Network
for epoch in range(num_epochs):
    train_loss, train_acc = [], []

    # bar = tqdm(dataloader['train'])
    bar = tqdm(train_loader)
    for batch_idx, (data, targets) in enumerate(bar):
    # for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # data, targets = batch.data, batch.targets
        # data, targets = data.to(device), targets.to(device)

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        # print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        print(scores.shape)
        print(targets.shape)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        train_loss.append(loss.item())
        acc = (targets == torch.argmax(scores, axis=1)).sum().item() / len(targets)
        train_acc.append(acc)
        bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")


loss 3.84292 acc 0.02511:  50%|█████     | 1/2 [01:03<01:03, 63.62s/it]

torch.Size([500000, 47])
torch.Size([500000])


  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([197932, 47])
torch.Size([197932])


loss 3.60705 acc 0.17221:  50%|█████     | 1/2 [01:02<01:02, 62.47s/it]

...

torch.Size([197932, 47])
torch.Size([197932])


loss 3.02321 acc 0.37711:  50%|█████     | 1/2 [01:02<01:02, 62.00s/it]

torch.Size([500000, 47])
torch.Size([500000])


loss 2.98402 acc 0.39299: 100%|██████████| 2/2 [01:26<00:00, 43.09s/it]

torch.Size([197932, 47])
torch.Size([197932])


In [8]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 42.60
Accuracy on test set: 42.48
